In [12]:
# Prep ... import libraries
import pandas as pd
import requests
from pathlib import Path

# Import API key & set base URL
from api_keys import walkscore_key
url="https://api.walkscore.com/score?format=json&address=skip"

# Define variables
FindState = "OH"

In [13]:
# Import CSV
data_df = pd.read_csv("../resources/obesity_data_ALL_states.csv")
data_df = pd.DataFrame(data_df)

# Insert new column at end to import Walk Score
data_df["Walk Score"]=''

# Clean up dataframe by removing any rows with no data
data_df_cleaned=data_df.dropna()

# data_df_cleaned["StateAbbr"].value_counts()

In [14]:
# Let's do API calls for just ONE state
# We define FindState in first block...
one_state = data_df_cleaned.loc[data_df_cleaned["StateAbbr"] == FindState, :]

# reset index because of dropped rows
one_state.reset_index(drop=True, inplace=True)

one_state.head(10)

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,OH,Parma,3961000-39035177404,3961000,39035177404,29.2,2875,41.389745,-81.739448,
1,OH,Columbus,3918000-39049000810,3918000,39049000810,37.8,2540,40.026543,-82.976199,
2,OH,Columbus,3918000-39049008825,3918000,39049008825,35.1,5908,39.886204,-82.977271,
3,OH,Toledo,3977000-39095008400,3977000,39095008400,32.2,758,41.667167,-83.683257,
4,OH,Dayton,3921000-39113165200,3921000,39113165200,48.8,1669,39.767184,-84.198173,
5,OH,Cleveland,3916000-39035101101,3916000,39035101101,47.6,1783,41.481730,-81.748051,
6,OH,Akron,3901000-39153507101,3901000,39153507101,31.3,5195,41.125325,-81.576844,
7,OH,Cincinnati,3915000-39061001100,3915000,39061001100,42.8,900,39.109816,-84.506063,
8,OH,Cincinnati,3915000-39061007400,3915000,39061007400,36.3,1598,39.162343,-84.534829,
9,OH,Canton,3912000-39151712202,3912000,39151712202,28.4,2030,40.852361,-81.355221,


In [15]:
# Now let's iterate thru that ONE state and get the data

for index,row in one_state.iterrows():

    try:
        lat = str(row["Lat"])
        lon = str(row["Lon"])
        query_url = url+"&lat="+lat+"&lon="+lon+"&wsapikey="+walkscore_key
        walk_response = requests.get(query_url)
        walk_json = walk_response.json() 
        if walk_json["status"] == 41:
            print("Daily API quota exceeded")
            print("^^^^^^^^^^^^^^^^^^^^^^^^")
            break
            
        print("Processing LAT %s, LON, %s" % (lat, lon))
        one_state.loc[index, "Walk Score"] = walk_json["walkscore"]
    
    except:
        print("Data not found. Skipping")
        pass


Processing LAT 41.3897449179, LON, -81.7394479009
Processing LAT 40.0265433635, LON, -82.9761993151
Processing LAT 39.8862043104, LON, -82.9772707362
Processing LAT 41.6671665767, LON, -83.6832573727
Processing LAT 39.7671842175, LON, -84.1981734091
Processing LAT 41.4817300513, LON, -81.7480506509
Processing LAT 41.1253246667, LON, -81.5768443398
Processing LAT 39.1098158418, LON, -84.5060626783
Processing LAT 39.1623431656, LON, -84.5348285925
Processing LAT 40.8523614138, LON, -81.3552209996
Processing LAT 41.4355398318, LON, -81.5724890211
Processing LAT 41.5021684751, LON, -81.6091133657
Processing LAT 40.7944191553, LON, -81.3882382803
Processing LAT 41.0654771085, LON, -81.5056081414
Processing LAT 41.0518660947, LON, -81.5404127107
Processing LAT 39.1107387959, LON, -84.6858320913
Processing LAT 40.038159335, LON, -82.9834210035
Processing LAT 39.9303449052, LON, -82.9428810346
Processing LAT 39.1612206526, LON, -84.4258173747
Processing LAT 41.3573881504, LON, -81.7473845645
P

In [10]:
one_state.head()

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,CA,Los Angeles,0644000-06037190902,644000,6037190902,30.2,3960,34.094395,-118.319013,86
1,CA,Stockton,0675000-06077003305,675000,6077003305,26.2,3704,38.001905,-121.307265,69
2,CA,Corona,0616350-06065048200,616350,6065048200,24.7,4301,33.888935,-117.540737,7
3,CA,Los Angeles,0644000-06037211122,644000,6037211122,24.8,3075,34.072679,-118.289223,91
4,CA,San Francisco,0667000-06075010700,667000,6075010700,14.8,5592,37.798514,-122.410047,100


In [11]:
# Save output to CSV
output_csv = f'../resources/{FindState}_complete.csv'
one_state.to_csv(output_csv, index=False)